# Pushing Block Simulation

Importing packages

In [1]:
try: 
    from jax import config
    config.update("jax_enable_x64", True)
    import time
    import jax.numpy as np
    from jax import jacfwd
    from viewer import Viewer1D

    print('Imported packages.')
except Exception as e:
    print('Importing packages failed:')
    print(e)



Imported packages.


Integrator and constants.

In [2]:
R = 0.1 # meter
m_block = 1.0 # kg
a_block = 1.0 # meter
q_init = [0., 2.0] # end effector, block : center x pos in meters
qdot_init = [0.5, 0.] # end effector, block : x vel in meters/dt


# Runge-Kutta 4
def rk4_step(f, x, dt):
    """
        Input:
            f(x) - function to be integrated
            x - current state
            dt - time step 
        Output: 
            x[t+dt] 
    """
    k1 = f(x)
    k2 = f(x + dt*k1/2)
    k3 = f(x + dt*k2/2)
    k4 = f(x + dt*k3)
    xdot = x + 1/6*(k1+2*k2+2*k3+k4)*dt
    return xdot 

Contact jacobian

In [7]:
def phi(q):
    return abs(q[0] - q[1]) - (R + a_block / 2)

J_c = jacfwd(phi)

The following function computes the dynamics of the system each timestep.

In [4]:
def dyn_step(x):
    """
        Input: 
            state = x = [q, qdot] 
        Output: 
            xdot = [qdot, qddot]
    """
    q, qdot = np.split(x, 2)
    # TODO
    qddot = [0., 0.]
    return np.hstack(np.array([qdot, qddot]))

Initialize the visualizer.

In [5]:
viewer = Viewer1D(R, a_block, q_init)
viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


## Animation

Using RK4 for integration to propagate.

In [6]:
tf = 4
dt = 0.01
N = int(tf/dt)
x0 = np.array(q_init + qdot_init)
for k in range(N):
    x0 = rk4_step(dyn_step, x0, dt)
    q,qdot = np.split(x0, 2)
    viewer.render(q)
    time.sleep(dt)